<a href="https://colab.research.google.com/github/lk2521/simple-chatbot/blob/main/basic_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM, Bidirectional, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'Is anyone there?',
    'Hello',
    'Hay',
    "What's up?",
    'Yo',
    'Greetings'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'Byy',
    'See you later',
    'Goodbye',
    'Farewell',
    'Take care'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help',
    'Appreciate it'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?',
    'What are you?',
    'Who you are?',
    'Tell me about yourself'],
   'responses': ["I'm Jarvis, your bot assistant",
    "I'm Jarvis, an Artificial Intelligent Bot"]},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?',
    'Tell me your nam

In [4]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [5]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 bidirectional (Bidirection  (None, 20, 128)           41472     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 12)                396       
                                                                 
Total params: 101164 (395.17 KB)
Trainable params: 10116

In [9]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 8s 160ms/step - loss: 2.4862 - accuracy: 0.1228
Epoch 2/500
2/2 [==============================] - 0s 17ms/step - loss: 2.4828 - accuracy: 0.1053
Epoch 3/500
2/2 [==============================] - 0s 156ms/step - loss: 2.4813 - accuracy: 0.1930
Epoch 4/500
2/2 [==============================] - 0s 162ms/step - loss: 2.4789 - accuracy: 0.1579
Epoch 5/500
2/2 [==============================] - 0s 16ms/step - loss: 2.4765 - accuracy: 0.1754
Epoch 6/500
2/2 [==============================] - 0s 160ms/step - loss: 2.4728 - accuracy: 0.1404
Epoch 7/500
2/2 [==============================] - 0s 156ms/step - loss: 2.4702 - accuracy: 0.1404
Epoch 8/500
2/2 [==============================] - 0s 163ms/step - loss: 2.4655 - accuracy: 0.1404
Epoch 9/500
2/2 [==============================] - 0s 158ms/step - loss: 2.4600 - accuracy: 0.1404
Epoch 10/500
2/2 [==============================] - 0s 15ms/step - loss: 2.4559 - accuracy: 0.1404
Epoch 11/500

In [10]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
!pip install colorama

In [15]:
import colorama
colorama.init()
from colorama import Fore, Style, Back

import random

def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
      print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
      inp = input()
      if inp.lower() == "quit":
          break

      result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                            truncating='post', maxlen=max_len))
      tag = lbl_encoder.inverse_transform([np.argmax(result)])

      intent_found = False

      for i in data['intents']:
          if i['tag'] == tag:
              print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))
              intent_found = True
              break

      if not intent_found:
          error_responses = [response for intent in data['intents'] if intent['tag'] == 'error' for response in intent['responses']]
          print(Fore.RED + "ChatBot:" + Style.RESET_ALL, random.choice(error_responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()


Start messaging with the bot (type quit to stop)!
User: Hello Friend
1/1 [==============================] - 2s 2s/step
ChatBot: Hello
User: who are you
1/1 [==============================] - 0s 24ms/step
ChatBot: I'm Jarvis, an Artificial Intelligent Bot
User: How's the weather?
1/1 [==============================] - 0s 22ms/step
ChatBot: I'm sorry, I don't have real-time weather information. You can check a reliable weather website or app for the latest updates.
User: Recommend some music
1/1 [==============================] - 0s 23ms/step
ChatBot: I recommend checking out 'Shape of You' by Ed Sheeran. It's a popular choice!
User: I want to raise a complaint
1/1 [==============================] - 0s 29ms/step
ChatBot: Please mention your complaint, we will reach you and sorry for any inconvenience caused
User: That's helpful
1/1 [==============================] - 0s 28ms/step
ChatBot: You're most welcome!
User: Goodbye
1/1 [==============================] - 0s 21ms/step
ChatBot: Have 